In [1]:
import datetime
import os
import backtrader as bt

for custom sizer:
- subclass bt.Sizer
- override method: _getsizing(self, comminfo, cash, data, isbuy)

In [2]:
class ArbitrarySizer(bt.Sizer):
    '''arbitrarily sizes trade based on the current day of month of the order and portfolio size'''

    def _getsizing(self, comminfo, cash, data, isbuy):
        day_of_month = bt.num2date(data.datetime[0]).day
        percent = day_of_month/100
        value = self.broker.get_value()*percent
        size = value/data.close[0]
        return int(size)


In [3]:
class MyStrat(bt.Strategy):
        
    def next(self):

        if not self.positionsbyname['spy']:
            self.buy(data=self.dnames['spy'], size=None) # sizer used when size=None (default)
        elif not self.positionsbyname['gld']:
            self.buy(data=self.dnames['gld'])
        elif not self.positionsbyname['uso']:
            self.buy(data=self.dnames['uso'])

In [4]:
cerebro = bt.Cerebro()

cerebro.addstrategy(MyStrat)
cerebro.addsizer(ArbitrarySizer)
fromdate, todate = datetime.datetime(2019, 6, 13), datetime.datetime(2019, 6, 20)
folder = 'ETF'
for file in os.listdir(folder):
    dn = os.path.join(folder, file)
    data = bt.feeds.GenericCSVData(
        dataname=dn,
        fromdate=fromdate,
        todate=todate,
        nullvalue=0.0,
        dtformat=('%m/%d/%Y'),
        datetime=0,
        high=3,
        low=4,
        open=1,
        close=2,
        volume=5,
        openinterest=-1
    )
    cerebro.adddata(data)

cerebro.addanalyzer(bt.analyzers.Transactions)
cerebro.broker.setcash(10000)
cerebro.broker.set_coc(True)
results = cerebro.run()
strat = results[0]
cerebro.broker.getvalue()

10107.59

In [5]:
for ticker in ['spy', 'gld', 'uso']:
    pos = strat.positionsbyname[ticker]
    price = strat.dnames[ticker].close[0]
    s = f'''{ticker.upper()} \nshares: {pos.size}\nvalue: {pos.size*price}\n% of portfolio: {round(100*pos.size*price/strat.broker.getvalue())}%\n'''
    print(s)

SPY 
shares: 4
value: 1172.24
% of portfolio: 12%

GLD 
shares: 11
value: 1406.79
% of portfolio: 14%

USO 
shares: 19
value: 1722.16
% of portfolio: 17%

